# Google Research Football

Exploration of *Rule Based and Reinforcement Learning Hybrid solution* for the Kaggle Competition: **Google Research Football with Manchester City F.C.**

The approach is as follows, decompose the agent into specific cases (like a freekick or a 1v1) and use RL agents trained on custom scenarios for optimal decison making.

# Installs

In [ ]:
%%bash
# Kaggle environments.
git clone https://github.com/Kaggle/kaggle-environments.git
cd kaggle-environments && pip install .

# GFootball environment.
apt-get update -y
apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
git clone -b v2.6 https://github.com/google-research/football.git
mkdir -p football/third_party/gfootball_engine/lib
wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.6.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so

# Custom Scenario

The game already has pre-built scenarios ([Repository](https://github.com/google-research/football/tree/master/gfootball/scenarios)). The Environment enables us to build custom scenarios. A scenario is a football game with specific characteristics such a limited number of players. For the scenario to be included in the environment, we must write into a file as such and build the game afterwards. The following gist shows how to create a custom scenario. 

**Select the options to customize (not all are needed) and change the variable (in capital letters), including the file name, to be the desired parameters.**

```python
%%writefile kaggle-environments/football/gfootball/scenarios/CUSTOM_NAME.py
# Imports 
from . import *

def build_scenario(builder):
    # Game Duration
    builder.config().game_duration = INTEGER
    builder.config().second_half = INTEGER
    
    # Default team decison making
    builder.config().right_team_difficulty = FLOAT #(Between 0 and 1)
    builder.config().left_team_difficulty = 1.0
    
    # ?
    builder.config().deterministic = BOOLEAN
    
    # Determing which side to be on (Optional)
    # If using this, set the teams accordingly: builder.SetTeam(first_team)
    if builder.EpisodeNumber() % 2 == 0:
        first_team = Team.e_Left
        second_team = Team.e_Right
    else:
        first_team = Team.e_Right
        second_team = Team.e_Left
    
    # Consider Offside Rule
    builder.config().offsides = BOOLEAN # Default = True
    
    # Conditions to end episodes
    builder.config().end_episode_on_score = BOOLEAN # Default = False
    builder.config().end_episode_on_out_of_play = BOOLEAN # Default = False
    builder.config().end_episode_on_possession_change = BOOLEAN # Default = False
    
    # Custom ball position
    builder.SetBallPosition(XPOS, YPOS)
    
    # Create Left Team
    builder.SetTeam(Team.e_Left)
    builder.AddPlayer(X, Y, ROLE, LAZY, CONTROLLABLE) 
    # x: x coordinate of the player in the range [-1, 1].
    # y: y coordinate of the player in the range [-0.42, 0.42].
    # role: Player's role in the game (goal keeper etc.).
    # lazy (Optional BOOLEAN default False): Computer doesn't perform any automatic actions for lazy player.
    # controllable (Optional BOOLEAN default True): Whether player can be controlled.

    # Create Right Team
    builder.SetTeam(Team.e_Right)
    builder.AddPlayer(-1.0, 0.0, e_PlayerRole_GK,)
    builder.AddPlayer(-0.7, 0.0, e_PlayerRole_CB)
```

In [ ]:
# This variable can be used for fancy animation.
# Make sure its value is the same as (or at least smaller than) the one in the scenario. 
game_duration = 300

In [ ]:
%%writefile kaggle-environments/football/gfootball/scenarios/custom_scenario_1v1.py
from . import *

def build_scenario(builder):
    builder.config().game_duration = 300
    builder.config().deterministic = False
    builder.config().offsides = False
    builder.config().end_episode_on_score = True
    builder.config().end_episode_on_out_of_play = True
    builder.config().end_episode_on_possession_change = True
    builder.SetBallPosition(0.5, 0.0)
    
    builder.SetTeam(Team.e_Left)
    builder.AddPlayer(-1.0, 0.0, e_PlayerRole_GK)
    builder.AddPlayer(0.1, -0.1, e_PlayerRole_CF)

    builder.SetTeam(Team.e_Right)
    builder.AddPlayer(-1.0, 0.0, e_PlayerRole_GK)
    builder.AddPlayer(-0.7, 0.05, e_PlayerRole_CB)

Now we can build the environment with the custom scenarios built into it.

In [ ]:
%%bash
# Install
cd kaggle-environments/football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
%%bash
# Proof of Concept: verify the presence of the desired files
find / -name custom_scenario*.py | grep site-packages | cut -c -90

# Bot
[Source](https://www.kaggle.com/piotrstanczyk/gfootball-template-bot)

In [ ]:
from kaggle_environments.envs.football.helpers import *
import numpy as np
from scipy.spatial.distance import cdist

constants = {
    "delta_x": 9.2e-3,  # How much player / ball position changes when moving without sprinting
    "sprinting_delta_x": 1.33e-2,  # How much player / ball position changes when sprinting
    "max_distance_to_influence_ball": 0.01,
    "max_distance_to_influence_ball_sprinting": 0.0185,
    "goalie_out_of_box_dist": 0.3,
    "on_breakaway_goalie_dist": 0.1
}

state = {
    "prev_owner": None,
    "count": 10,
    "prev_player_pos": [0, 0],
    "prev_ball_pos": [0, 0, 0]
}

dir_lookup = {
    "vectors": np.array([[1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0], [-1, 1], [0, 1], [1, 1]]),
    "actions": [Action.Right, Action.TopRight, Action.Top, Action.TopLeft, Action.Left, Action.BottomLeft, Action.Bottom, Action.BottomRight]
}


def find_closest_opposition_to_ball(ball_pos, active_players):
    '''Returns index of active player closest to the ball'''
    pass


def pass_path():
    pass


def move_to_position(pos, player_pos):
    '''
    Calculates vector between pos and player_pos, calculates cosine similarity to
    find the best direction action to get the player to the desired location in
    straight line fashion
    Args:
        pos: Desired destination position
        player_pos: Current player position
    Returns: Direction action
    '''
    if (len(pos) == 3):
        # 3D vector, convert to 2D
        pos = pos[:2]
    dir_array = dir_lookup["vectors"]
    desired_dir = (np.array(pos) - np.array(player_pos)).reshape(1, -1)

    # Map direction to closest action using cosine similarity
    cosine_dist = 1 - cdist(desired_dir, dir_array, metric='cosine')
    max_index = np.argmax(cosine_dist)
    return dir_lookup["actions"][max_index]

def on_breakaway(player_pos, obs):
    '''Detects if a player is on a breakaway or not'''
    player_x = player_pos[0]
    right_team_pos = obs["right_team"][1:] # Not including the goalie
    return all([player_x > r_ply_pos[0] for r_ply_pos in right_team_pos])


@human_readable_agent
def agent(obs):
    player_pos = obs['left_team'][obs['active']]
    prev_owner = state["prev_owner"]
    ball_owned_team = obs["ball_owned_team"]
    state["prev_owner"] = ball_owned_team
    ball_pos = obs["ball"]
    ball_direction = obs["ball_direction"]
    ball_owned_player = obs["ball_owned_player"]

    if Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint

    follow_ball_action = move_to_position(ball_pos, player_pos)
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        # Player is attacking

        # Calculate goalie position, if goalie presses could lead to loss of possesion
        goalie_pos = np.array(obs["right_team"][0])
        dist_to_goalie = np.linalg.norm(np.array(player_pos) - goalie_pos)
        dist_goalie_off_line = np.linalg.norm(np.array([1, 0]) - goalie_pos)

        # If we are on a breakaway and the goalie has come close, ideally we should move around goalie
        if on_breakaway(player_pos, obs):
            if dist_to_goalie < constants["on_breakaway_goalie_dist"]:
                return Action.Shot

        # If goalie is really far out of the box worth trying a shot
        if dist_goalie_off_line > constants["goalie_out_of_box_dist"]:
            return Action.Shot

        # Shot if we are 'close' to the goal (based on 'x' coordinate).
        if player_pos[0] > 0.5:
            return Action.Shot

        # Run towards the goal otherwise.
        return Action.Right
    elif ball_owned_team == 1:
        # If the other team owns the ball
        opponent_pos = np.array(obs["right_team"][ball_owned_player])
        opponent_dir = np.array(obs["right_team_direction"][ball_owned_player])
        dist_to_ball = np.linalg.norm(opponent_pos - np.array(ball_pos[:2]))

        # If the player is close enough to change direction of the ball we should go to the new ball position
        if constants["max_distance_to_influence_ball_sprinting"] > dist_to_ball and np.linalg.norm(opponent_dir) != 0:
            # Need to make it a 2 vector as ball position 3 vector
            calc_ball_pos = np.array(ball_pos[:2])
            # Normalize their direction, opponent can change the ball dir by sprinting_delta_x each touch
            opponent_dir = opponent_dir / np.linalg.norm(opponent_dir)
            # Calculate how much the ball position will be changing in each direction
            ball_delta = constants["sprinting_delta_x"] * opponent_dir
            calc_ball_pos = calc_ball_pos + ball_delta

            # Return a new action that goes to where the ball should be going
            return move_to_position(calc_ball_pos, player_pos)

    state["prev_player_pos"] = player_pos
    state["prev_ball_pos"] = ball_pos

    # Just follow the ball
    return follow_ball_action

In [2]:
ar = [0,1,2,3,4,5,6,7,8,9,10,11]
ar[0:9]

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
%%writefile submission.py
from kaggle_environments.envs.football.helpers import *
import math

@human_readable_agent
def agent(obs):

    def distance(player_one, player_two):
        # Compute the distance between two players
        return math.hypot(player_one[0] - player_two[0], player_one[1] - player_two[1])
    
    def closest_player(controlled_player_position, team = 'left_team'):
    # Find the coordinates of the player closest to the player in posession
    # Use team = 'right_team' to get the closest opponent
    distances = obs[team].copy()
    for i, teammate_position in enumerate(obs[team]):
        distances[i] = distance(controlled_player_position, teammate_position)
    return np.argmin(distances)

    def coordinate_to_direction(coord_one, coord_two):
        # Given coordinate between a player and anything (player, goal ...), output cardinal direction 
        
        pass
        
    
    def direction_check(action, direction)
        # Sticky is a N hot encoded array for whether the player is performing an action
        # Here, we check that the player is directed properly or we let it perform an action
        if direction in obs['sticky_actions'][0:8]:
            return action
        else:
            return direction
    
    def expected_goal(player):
        # Compute expected goals from paper :https://www.researchgate.net/publication/240641737_Estimating_the_probability_of_a_shot_resulting_in_a_goal_The_effects_of_distance_angle_and_space
        # y = 0.337 - 0.157 * (distance) - 0.022 * (angle) + 0.799 * (space)
        goal = [1, 0]

        dist = lambda other: math.hypot(other[0] - player[0], other[1] - player[1])
        distanceFromGoal = 1 / dist(goal) # Smaller distances are better
        angleFromGoal = 1 / abs(math.degrees(math.atan2(goal[1] - player[1], goal[0] - player[0]))) # Smaller angles are better

        enemyDistances = list(map(dist, obs['right_team']))
        closestEnemyDistance = min(enemyDistances) / 2 # Larger distances are better

        return (15 * distanceFromGoal) + (2 * angleFromGoal) + (200 * closestEnemyDistance) # Weighted
    
    # Get the position of the players to be controlled    
    # We always control left team (observations and actions
    # are mirrored appropriately by the environment).
    controlled_player_position = obs['left_team'][obs['active']]
    
    # Offense
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        if obs["game_mode"] == 1:
            # KickOff
            return Action.ShortPass, 
        elif obs["game_mode"] == 2:
            # GoalKick
            return Action.LongPass
        elif obs["game_mode"] == 3:
            # FreeKick
            return Action.Shot
        elif obs["game_mode"] == 4:
            # Corner
            return Action.HighPass
        elif obs["game_mode"] == 5:
            # ThrowIn
            return Action.ShortPass
        elif obs["game_mode"] == 6:
            # Penalty
            return Action.Shot
        else:
            # Normal
             if controlled_player_position[0] > 0.6:
                return Action.Shot
            # Run towards the goal otherwise.
            return Action.Right
    # Defense
    else:
        # Run towards the ball.
        if obs['ball'][0] > controlled_player_position[0] + 0.05:
            return Action.Right
        if obs['ball'][0] < controlled_player_position[0] - 0.05:
            return Action.Left
        if obs['ball'][1] > controlled_player_position[1] + 0.05:
            return Action.Bottom
        if obs['ball'][1] < controlled_player_position[1] - 0.05:
            return Action.Top
        # Try to take over the ball if close to the ball.
        return Action.Slide

# Environment Setup

Now we can initialize the environment with the desired scenario (ours or the pre-built ones) and use our agent on it.

In [ ]:
from kaggle_environments import make

scenarios = {0: "11_vs_11_competition",
             1: "11_vs_11_easy_stochastic",
             2: "11_vs_11_hard_stochastic",
             3: "11_vs_11_kaggle",
             4: "11_vs_11_stochastic",
             5: "1_vs_1_easy",
             6: "5_vs_5",
             7: "academy_3_vs_1_with_keeper",
             8: "academy_corner",
             9: "academy_counterattack_easy",
             10: "academy_counterattack_hard",
             11: "academy_empty_goal",
             12: "academy_empty_goal_close",
             13: "academy_pass_and_shoot_with_keeper",
             14: "academy_run_pass_and_shoot_with_keeper",
             15: "academy_run_to_score",
             16: "academy_run_to_score_with_keeper",
             17: "academy_single_goal_versus_lazy",
             18: "custom_scenario_1v1"
             }

# Choose here the scenario of interest
env = make("football", debug=True,
           configuration={"save_video": True, 
                          "scenario_name": scenarios[0], 
                          "running_in_notebook": True})
         
# Choose here the agents that play
output = env.run(["/kaggle/working/submission.py", "do_nothing"])
scores = output[-1][0]["observation"]["players_raw"][0]["score"]
print("Scores  {0} : {1}".format(*scores))

print('Left player: reward = %s, status = %s, info = %s' % (output[-1][0]['reward'], output[-1][0]['status'], output[-1][0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[-1][1]['reward'], output[-1][1]['status'], output[-1][1]['info']))

# Optional default Game Rendering
env.render(mode="human", width=600, height=400)

# Fancy Animation
[Source](https://www.kaggle.com/jaronmichal/human-readable-visualization)

In [ ]:
import matplotlib.patches as patches
from  matplotlib.patches import Arc
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib.patches as mpatches

from enum import Enum

import math

import numpy as np
from IPython.display import HTML

# Change size of figure
plt.rcParams['figure.figsize'] = [20, 16]
# Function that draw the green pitch with white lines
def drawPitch(width, height, color="w"):    
    fig = plt.figure()
    ax = plt.axes(xlim=(-10, width + 10), ylim=(-15, height + 5))
    plt.axis('off')

    # Grass around pitch
    rect = patches.Rectangle((-5,-5), width + 10, height + 10, linewidth=1, edgecolor='gray',facecolor='#3f995b', capstyle='round')
    ax.add_patch(rect)

    # Pitch boundaries
    rect = plt.Rectangle((0, 0), width, height, ec=color, fc="None", lw=2)
    ax.add_patch(rect)

    # Middle line
    plt.plot([width/2, width/2], [0, height], color=color, linewidth=2)

    # Dots
    dots_x = [11, width/2, width-11]
    for x in dots_x:
        plt.plot(x, height/2, 'o', color=color, linewidth=2)

    # Penalty box  
    penalty_box_dim = [16.5, 40.3]
    penalty_box_pos_y = (height - penalty_box_dim[1]) / 2

    rect = plt.Rectangle((0, penalty_box_pos_y), penalty_box_dim[0], penalty_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((width, penalty_box_pos_y), -penalty_box_dim[0], penalty_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)

    #Goal box
    goal_box_dim = [5.5, penalty_box_dim[1] - 11 * 2]
    goal_box_pos_y = (penalty_box_pos_y + 11)

    rect = plt.Rectangle((0, goal_box_pos_y), goal_box_dim[0], goal_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((width, goal_box_pos_y), -goal_box_dim[0], goal_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)

    #Goals
    rect = plt.Rectangle((0, penalty_box_pos_y + 16.5), -3, 7.5, ec=color, fc=color, lw=2, alpha=0.3)
    ax.add_patch(rect)
    rect = plt.Rectangle((width, penalty_box_pos_y + 16.5), 3, 7.5, ec=color, fc=color, lw=2, alpha=0.3)
    ax.add_patch(rect)

    # Middle circle
    mid_circle = plt.Circle([width/2, height/2], 9.15, color=color, fc="None", lw=2)
    ax.add_artist(mid_circle)


    # Penalty box arcs
    left  = patches.Arc([11, height/2], 2*9.15, 2*9.15, color=color, fc="None", lw=2, angle=0, theta1=308, theta2=52)
    ax.add_patch(left)
    right = patches.Arc([width - 11, height/2], 2*9.15, 2*9.15, color=color, fc="None", lw=2, angle=180, theta1=308, theta2=52)
    ax.add_patch(right)

    # Arcs on corners
    corners = [[0, 0], [width, 0], [width, height], [0, height]]
    angle = 0
    for x,y in corners:
        c = patches.Arc([x, y], 2, 2, color=color, fc="None", lw=2, angle=angle,theta1=0, theta2=90)
        ax.add_patch(c)
        angle += 90
    return fig, ax

WIDTH = 105
HEIGHT = 68
X_RESIZE = WIDTH
Y_RESIZE = HEIGHT / 0.42

class GameMode(Enum):
    Normal = 0
    KickOff = 1
    GoalKick = 2
    FreeKick = 3
    Corner = 4
    ThrowIn = 5
    Penalty = 6

def scale_x(x):
    return (x + 1) * (X_RESIZE/2)

def scale_y(y):
    return (y + 0.42) * (Y_RESIZE/2)

def extract_data(frame):
    # Function that gets game data at each time frame
    res = {}
    obs = frame[0]['observation']['players_raw'][0]
    res["left_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["left_team"]]
    res["right_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["right_team"]]

    ball_x, ball_y, ball_z = obs["ball"]
    res["ball"] = [scale_x(ball_x),  scale_y(ball_y), ball_z]
    res["score"] = obs["score"]
    res["steps_left"] = obs["steps_left"]
    res["ball_owned_team"] = obs["ball_owned_team"]
    res["ball_owned_player"] = obs["ball_owned_player"]
    res["right_team_roles"] = obs["right_team_roles"]
    res["left_team_roles"] = obs["left_team_roles"]
    res["left_team_direction"] = obs["left_team_direction"]
    res["right_team_direction"] = obs["right_team_direction"]
    res["game_mode"] = GameMode(obs["game_mode"]).name
    return res

def draw_team(obs, team, side):
    X = []
    Y = []
    for x, y in obs[side]:
        X.append(x)
        Y.append(y)
    team.set_data(X, Y)

def draw_ball(obs, ball):
    # Function that draws the ball on the field    
    ball.set_markersize(10 + 5 * obs["ball"][2]) # Scale size of ball based on height
    ball.set_data(obs["ball"][:2])

def draw_ball_owner(obs, ball_owner, team_active):
    # Function that highlights the player in possession of the ball    
    if obs["ball_owned_team"] == 0:
        x, y = obs["left_team"][obs["ball_owned_player"]]
        ball_owner.set_data(x, y)
        team_active.set_data(WIDTH / 4 + 7, -7)
        team_active.set_markerfacecolor("red")
    elif obs["ball_owned_team"] == 1:
        x, y = obs["right_team"][obs["ball_owned_player"]]
        ball_owner.set_data(x, y)
        team_active.set_data(WIDTH / 4 + 50, -7)
        team_active.set_markerfacecolor("blue")
    else:
        ball_owner.set_data([], [])
        team_active.set_data([], [])
    
def draw_players_directions(obs, directions, side):
    # Function that direction in which players run
    index = 0
    if "right" in side:
        index = 11
    for i, player_dir in enumerate(obs[f"{side}_direction"]):
        x_dir, y_dir = player_dir
        dist = math.sqrt(x_dir ** 2 + y_dir ** 2) + 0.00001 # to prevent division by 0
        x = obs[side][i][0]
        y = obs[side][i][1] 
        directions[i + index].set_data([x, x + x_dir / dist ], [y, y + y_dir / dist])

# Settup for the animation
fig, ax = drawPitch(WIDTH, HEIGHT)
ax.invert_yaxis()

ball_owner, = ax.plot([], [], 'o', markersize=30,  markerfacecolor="yellow", alpha=0.5)
team_active, = ax.plot([], [], 'o', markersize=30,  markerfacecolor="blue", markeredgecolor="None")

team_left, = ax.plot([], [], 'o', markersize=20, markerfacecolor="r", markeredgewidth=2, markeredgecolor="white")
team_right, = ax.plot([], [], 'o', markersize=20,  markerfacecolor="b", markeredgewidth=2, markeredgecolor="white")

ball, = ax.plot([], [], 'o', markersize=10,  markerfacecolor="black", markeredgewidth=2, markeredgecolor="white")
text_frame = ax.text(-5, -5, '', fontsize=25)
match_info = ax.text(105 / 4 + 10, -5, '', fontsize=25)
game_mode = ax.text(105 - 25, -5, '', fontsize=25)
goal_notification = ax.text(105 / 4 + 10, 0, '', fontsize=25)

# Drawing of directions definitely can be done in a better way
directions = []
for i in range(22):
    direction, = ax.plot([], [], color='yellow', lw=3)
    directions.append(direction)

drawings = [team_active, ball_owner, team_left, team_right, ball, text_frame, match_info, game_mode, goal_notification]

def init():
    team_left.set_data([], [])
    team_right.set_data([], [])
    ball_owner.set_data([], [])
    team_active.set_data([], [])
    ball.set_data([], [])
    return drawings 

def animate(i):
    global prev_score_a, prev_score_b
    obs = extract_data(output[i])

    # Draw info about ball possesion
    draw_ball_owner(obs, ball_owner, team_active)

    # Draw players
    draw_team(obs, team_left, "left_team")
    draw_team(obs, team_right, "right_team")

    draw_players_directions(obs, directions, "left_team")
    draw_players_directions(obs, directions, "right_team")

    draw_ball(obs, ball)

    # Draw textual informations
    text_frame.set_text(f"Frame: {i}/{obs['steps_left'] + i - 1}")
    game_mode.set_text(f"Game mode: {obs['game_mode']}")

    score_a, score_b = obs["score"]
    match_info.set_text(f"Left team {score_a} : {score_b} Right Team")

    return drawings  

In [ ]:
# Build the animation and draw it
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=game_duration, interval=100, blit=True)

HTML(anim.to_html5_video())